In [1]:
import cabi
import pandas as pd

# from numba import jit,uint16,vectorize,uint32

The goal of this notebook is to take the 29 million trips, count how many occur between each two stations, then sum then to find the undirected number of trips (ie all trips starting or ending at this station)

# Creating trip summary table
Our goal is to create a table showing the stations in a certain pair of stations
## Loading trip data
First we load the table of all trips, and remove any with incomplete data

In [2]:
# df = cabi.return_trip_datatable()
# df.to_parquet("../data/interim/comb_trips.gzip", compression="gzip")

In [3]:
df = pd.read_parquet("../data/interim/comb_trips.gzip")
total = len(df)
print(f"{total} total trips")

31543723 total trips


In [4]:
df

,started_at,ended_at,start_station_id,end_station_id,member_casual
0,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,31108,member
1,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,31209,member
2,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,31100,member
3,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,31602,member
4,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,31201,member
...,...,...,...,...,...
254245,2022-03-12 17:02:00,2022-03-12 17:11:09,31641,31629,member
254246,2022-03-19 15:28:33,2022-03-19 16:04:03,31641,31290,casual
254247,2022-03-29 13:08:30,2022-03-29 13:18:09,31660,31637,member
254248,2022-03-01 13:04:29,2022-03-01 13:11:42,31660,31637,member


In [5]:
# clean the NA values out
df = df.dropna()
total_nonna = len(df)
print(
    f"{total_nonna} after removing trips with NA values \n {total-total_nonna} trips removed due to NA values"
)

31081858 after removing trips with NA values 
 461865 trips removed due to NA values


## Removing self-trips
There are interesting but they aren't easy to show on a map


In [6]:
df = df[df.start_station_id != df.end_station_id]
total_nonself = len(df)
print(
    f"{total_nonself} trips starting and ending at the same station \n {total_nonna-total_nonself} trips were removed"
)

29690088 trips starting and ending at the same station 
 1391770 trips were removed


## Removing invalid trips
We have some number of trips where the end station is listed as 0. Obviously these are invalid

In [7]:
df = df[df.start_station_id > 0]
df = df[df.end_station_id > 0]
total_mappable = len(df)
print(
    f"{total_mappable} trips after removing trips with missing origin or destination \n {total_nonna-total_mappable} trips removed"
)

29688538 trips after removing trips with missing origin or destination 
 1393320 trips removed


## Removing trips from removed stations
We also need to make sure we know the location of the station. Therefore we load the list of stations names. Maybe in the future if I can find a table of the locations of removed stations and we can add them to the visualization
### loading our current stations

In [8]:
# Define which attributes to lookup from airports.csv
cabi_stations = "https://raw.githubusercontent.com/mlinds/cabi-data/main/data/processed/stationLookup.csv"
station_names_list = list(pd.read_csv(cabi_stations).short_name)

### Selecting only trips involving extant stations
We rewrite the dataframe to include only stations that exist in our location lookup table

In [9]:
df = df[df.end_station_id.map(lambda x: x in station_names_list)]
df = df[df.start_station_id.map(lambda x: x in station_names_list)]
total_current = len(df)
print(
    f"{total_current} trips after removing trips from stations that dont exist \n {total_mappable-total_current} trips were removed"
)

29356295 trips after removing trips from stations that dont exist 
 332243 trips were removed


In [10]:
df.dtypes

started_at          datetime64[ns]
ended_at            datetime64[ns]
start_station_id            UInt16
end_station_id              UInt16
member_casual             category
dtype: object

In [11]:
import numpy as np

# @jit(uint32(uint16,uint16))
def sort_stations(st, end):
    a, b = sorted([st, end])
    return int(str(a) + str(b))

In [12]:
# @jit
def sort_stations_dataframe(starray, endarray, ind):
    st = starray.iloc[ind]
    end = endarray.iloc[ind]
    return sort_stations(st, end)


sorter = lambda i: sort_stations_dataframe(
    sample_df.start_station_id, sample_df.end_station_id, i
)

In [13]:
sample_df = df[["start_station_id", "end_station_id"]].reset_index()

In [14]:
%%time
popularity = sample_df.groupby(by=sorter).count()

Wall time: 21min 18s


In [15]:
new_pop = popularity.start_station_id

In [16]:
indicies = pd.DataFrame(new_pop).index

In [17]:
stlist = [int(str(item)[0:5]) for item in indicies]
endlist = [int(str(item)[5:10]) for item in indicies]
pop_df = (
    pd.DataFrame(new_pop)
    .assign(st=stlist, en=endlist)
    .reset_index(drop=True)
    .rename(columns={"start_station_id": "popularity"})[["st", "en", "popularity"]]
)
pop_df.to_csv(
    "../data/processed/connections_csv.csv",
    columns=["st", "en", "popularity"],
    index=False,
)

In [18]:
pop_df.sort_values("popularity", ascending=False)

,st,en,popularity
49820,31247,31258,80237
50437,31249,31258,70888
75626,31613,31619,67595
76808,31623,31631,58213
50135,31248,31258,54030
...,...,...,...
75710,31613,31817,1
18673,31067,31301,1
41377,31217,31907,1
75713,31613,31823,1


In [19]:
all_stations = pd.read_csv(cabi_stations)
all_stations

,short_name,name,lat,lon
0,31000,Eads St & 15th St S,38.858971,-77.053230
1,31002,Crystal Dr & 20th St S,38.856425,-77.049232
2,31003,Crystal Dr & 15th St S,38.861056,-77.049417
3,31004,Aurora Hills Cmty Ctr / 18th St & S Hayes St,38.857866,-77.059490
4,31005,Pentagon City Metro / 12th St & S Hayes St,38.862303,-77.059936
...,...,...,...,...
647,31951,8th Rd & S Frederick St,38.857049,-77.116339
648,31531,MBT & Bryant St NE,38.922404,-76.996023
649,31952,N Howard St & N Imboden St,38.818509,-77.105564
650,32255,Hartland Rd & Harte Pl,38.878601,-77.222808


In [20]:
32232, 32600

(32232, 32600)

In [21]:
df

,started_at,ended_at,start_station_id,end_station_id,member_casual
0,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,31108,member
2,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,31100,member
3,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,31602,member
4,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,31201,member
5,2010-09-20 12:14:27,2010-09-20 12:30:50,31109,31200,member
...,...,...,...,...,...
254245,2022-03-12 17:02:00,2022-03-12 17:11:09,31641,31629,member
254246,2022-03-19 15:28:33,2022-03-19 16:04:03,31641,31290,casual
254247,2022-03-29 13:08:30,2022-03-29 13:18:09,31660,31637,member
254248,2022-03-01 13:04:29,2022-03-01 13:11:42,31660,31637,member


In [22]:
df.query("start_station_id == 32232 & end_station_id == 32600")

,started_at,ended_at,start_station_id,end_station_id,member_casual
122038,2019-05-13 18:49:40,2019-05-13 18:53:52,32232,32600,member
170670,2019-05-17 19:14:19,2019-05-17 19:52:15,32232,32600,casual
263803,2019-05-25 14:38:31,2019-05-25 14:42:11,32232,32600,member
331672,2019-05-31 15:50:49,2019-05-31 15:59:49,32232,32600,casual
331674,2019-05-31 15:50:50,2019-05-31 16:00:05,32232,32600,casual
...,...,...,...,...,...
31430,2022-02-01 13:19:46,2022-02-01 13:22:16,32232,32600,member
31538,2022-02-26 13:50:35,2022-02-26 13:55:49,32232,32600,member
187103,2022-03-01 19:09:54,2022-03-01 19:13:54,32232,32600,member
187104,2022-03-02 19:32:12,2022-03-02 19:37:20,32232,32600,member


In [23]:
df.query("start_station_id == 32600 & end_station_id == 32232")

,started_at,ended_at,start_station_id,end_station_id,member_casual
160525,2019-05-17 07:43:48,2019-05-17 07:46:31,32600,32232,member
331528,2019-05-31 15:38:07,2019-05-31 15:40:50,32600,32232,member
26894,2019-06-03 13:20:30,2019-06-03 13:23:59,32600,32232,casual
68216,2019-06-06 17:08:08,2019-06-06 17:36:17,32600,32232,casual
137071,2019-06-12 17:43:50,2019-06-12 17:49:40,32600,32232,member
...,...,...,...,...,...
9575,2022-01-15 17:51:16,2022-01-15 17:55:08,32600,32232,member
9581,2022-01-28 09:11:09,2022-01-28 09:15:59,32600,32232,member
61834,2022-01-29 12:57:54,2022-01-29 13:02:28,32600,32232,member
4954,2022-02-01 09:42:44,2022-02-01 09:45:23,32600,32232,member
